### Librerias

In [ ]:
import cv2
import os
import numpy as np

### Funcion Guardar Fotos de un Video

In [ ]:
def save_frames_from_video(video_path, output_folder, frame_size=(100, 100), frame_interval=10):
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        resized_frame = cv2.resize(frame, frame_size)

        frame_filename = f"frame_{frame_count:04d}.jpg"
        frame_path = os.path.join(output_folder, frame_filename)
        cv2.imwrite(frame_path, resized_frame)

        frame_count += 1

        # Intervalo
        for _ in range(frame_interval):
            ret, _ = cap.read()
            if not ret:
                break

    print(f"{frame_count} fotos guardadas en {output_folder}")
    cap.release()


### Funcion Para Aplicar el Haarcascade

In [ ]:
def buscando_a_nemo(video_path, cascade_path, scale_factor=1.1, min_neighbors=5):
    clasificador = cv2.CascadeClassifier(cascade_path)

    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect objects in the frame
        objects = clasificador.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=min_neighbors)

        # Draw rectangles around the detected objects
        for (x, y, w, h) in objects:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2.imshow('deteccion', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

### Obtener Imagenes Negativas

In [ ]:
nfolder = './faces/n'
nvideo = './negative.mp4'

if not os.path.exists(nfolder):
    os.makedirs(nfolder)
    print(f"Folder '{nfolder}' creado")

save_frames_from_video(nvideo, nfolder, frame_size=(46, 46), frame_interval=5)

### Obtener Imagenes Positivas

In [ ]:
pfolder = './faces/p'
pvideo = './positive.mp4'

if not os.path.exists(pfolder):
    os.makedirs(pfolder)
    print(f"Folder '{pfolder}' creado")

save_frames_from_video(pvideo, pfolder, frame_size=(46, 46), frame_interval=5)

### Generar Haarcasacde

In [ ]:
dataset = './faces'
dirlist = os.listdir(dataset)
print("Archivos: ", dirlist)

labels = []
facesData = []
label = 0

for namedir in dirlist:
    dirpath = dataset + "/" + namedir
    
    for  filename in os.listdir(dirpath):
        imagepath = dirpath + "/" + filename
        print(imagepath)
        image = cv2.imread(imagepath, 0)
        
        facesData.append(image)
        labels.append(label)
    label += 1
    
# print("Tag 0: ", np.count_nonzero(np.array(label) == 0))
# print("Tag 1: ", np.count_nonzero(np.array(label) == 1))

face_mask_train = cv2.face.LBPHFaceRecognizer_create()

print("Trainning...")
face_mask_train.train(facesData, np.array(labels))

face_mask_train.write("cubrebocas.xml")
print("modelo almacenado")

### Utilizar cascade.xml Generado

In [7]:
test_video = 'test.mp4'
# cascade_path = './classifier/cascade.xml'
# cascade_path = './haarcascade_frontalface_alt.xml'
cascade_path = './face_mask.xml'

if os.path.exists(cascade_path):
    # buscando_a_nemo(search_video, cascade_path, scale_factor=1.5, min_neighbors=6)
    mp_face_detection = mp.solution.face_detection
    
    LABELS = ["Con_mascarilla", "Sin_mascarilla"]
    
    face_mask = cv2.face.LBPHFaceRecognizer_create()
    face_mask.read(cascade_path)
    
    cap = cv2.VideoCapture(test_video)
    
    with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
        
        while True:
            ret, frame = cap.read()
            if ret == False: break
            
            cv2.imshow("Frame", frame)
            k = cv2.waitKey(1)
            if k == 27:
                break
            
    cap.release()
    cv2.destroyAllWindows()


AttributeError: module 'numpy' has no attribute 'solution'